# Hyperparameter Tuning
Find the most acceptable parameter

In [2]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy

## Data for Tuning

In [8]:
query = """SELECT 
                p.post_id,
                p.post_type,
                CASE
                    WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                    ELSE "buka"
                END AS is_operate
            FROM 
                posts p
            LEFT JOIN
                posts_jadwal pj
                ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
            """

df_location = sql_connection.read_from_sql(query)

C:\Users\rahma\Documents\kuliah\thesis\rahmat_code\travel-itinerary-recommendation\sql_connection.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql(query, mydb)


In [9]:
random.seed(10)
tourid = df_location[(df_location['post_type']=="location")&
                     (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
tourid = random.sample(tourid,30)
idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
idhotel = idhotel[random.randint(0,len(idhotel)-1)]
dwaktu,dtarif,drating = 1,1,1
travel_days = 3

In [10]:
db = ConDB()

hotel = db.HotelbyID(idhotel)
tur = db.WisatabyID(tourid)
timematrix = db.TimeMatrixbyID(hotel._id,tourid)

## Maximum Iteration

In [5]:
# List of iterations
iterations = [100, 200, 300]

#list of algorithms
for n in iterations:
    print(f"n = {n}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(max_iter = n,random_state=10)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("ACS - TSP")
    start = time.time()
    acs_tsp = ACS_TSP(max_iter = n,random_state=10)
    acs_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("BSO - VRP")
    start = time.time()
    bso_vrp = BSO_VRP(max_iter = n,two_opt_method = "best",random_state=10)
    bso_vrp.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = bso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("BSO - TSP")
    start = time.time()
    bso_tsp = BSO_TSP(max_iter = n,two_opt_method = "best",random_state=10)
    bso_tsp.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = bso_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = n,random_state=10)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = n,random_state=10)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = n,random_state=10)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = n,random_state=10)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = n,random_state=10)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("ACS BSO - TSP")
    start = time.time()
    acsbso_tsp = ACSBSO_TSP(max_iter_acs = n,random_state=10)
    acsbso_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")


n = 100
ACS - VRP
fitness = 0.6640717091850322
time = 3.145900249481201

ACS - TSP
fitness = 0.5713543998805943
time = 4.432577610015869

BSO - VRP
fitness = 0.5586581135660008
time = 0.775104284286499

BSO - TSP
fitness = 0.5145780744874009
time = 30.59467625617981

ACO - VRP
fitness = 0.6331323388066337
time = 4.2995452880859375

ACO - TSP
fitness = 0.5465923088404376
time = 4.123947620391846

HACO - VRP
fitness = 0.6373008579185134
time = 9.445233583450317

HACO - TSP
fitness = 0.5880791730604625
time = 9.643141031265259

ACS BSO - VRP
fitness = 0.6679080823438911
time = 47.68351721763611

ACS BSO - TSP
fitness = 0.4847424170106957
time = 129.83533263206482
n = 200
ACS - VRP
fitness = 0.6645953579459625
time = 10.81521987915039

ACS - TSP
fitness = 0.5758542908082344
time = 11.09622073173523

BSO - VRP
fitness = 0.5586581135660008
time = 0.7219836711883545

BSO - TSP
fitness = 0.5145780744874009
time = 59.64255976676941

ACO - VRP
fitness = 0.6331323388066337
time = 3.87328743934631

**results:**
- semua ACS-based bisa menggunakan max iter = 200
- BSO standalone bisa menggunakan max iter = 100
- ACO-based bisa menggunakan max iter = 100

## Tuning ACS (VRP/TSP)
Tuning dilakukan dengan VRP, TSP mengikuti

default parameter/parameter awal:
- alpha_t = 1
- beta = 2
- q0 = 0.1
- init_pheromone = 0.1
- rho = 0.1
- alpha = 0.1
- num_ant = 30

### Find Initial Pheromone

In [6]:
tur

[Tourism Zone Malioboro(00:00:00-00:00:00),
 Rumah Hantu Malioboro(00:00:00-00:00:00),
 The Lost World Castle(00:00:00-00:00:00),
 Tlogo Muncar National Garden(00:00:00-00:00:00),
 Timang Beach(00:00:00-00:00:00),
 Bukit Rhema Gereja Ayam(00:00:00-00:00:00),
 Jogja City Mall(00:00:00-00:00:00),
 De Mata Trick Eye Museum(00:00:00-00:00:00),
 Sindu Kusuma Edupark (SKE)(00:00:00-00:00:00),
 Seribu Batu Songgo Langit(00:00:00-00:00:00)]

In [35]:
#find init pheromone
init_pheromone = [0.1,0.3,0.5,1]

for pheromone in init_pheromone:
    print(f"pheromone = {pheromone}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(max_iter = 200,init_pheromone=pheromone,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

pheromone = 0.1
ACS - VRP
fitness = 0.6606055340691207
time = 4.665886640548706
pheromone = 0.3
ACS - VRP
fitness = 0.6694069251927628
time = 16.335801601409912
pheromone = 0.5
ACS - VRP
fitness = 0.657590153247745
time = 3.5124146938323975
pheromone = 1
ACS - VRP
fitness = 0.6696641268388532
time = 16.188631534576416


pheromone 0.3 atau 1 memberikan hasil yang baik tetapi lambat konvergen. acceptable value = 0.1 atau 0.3 atau 1

### Find Alpha_t

In [43]:
#find alpha_t
alpha_t_ls = [1,2,3,4,5]

for alpha_t in alpha_t_ls:
    print(f"alpha_t = {alpha_t}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(init_pheromone=0.1,alpha_t = alpha_t,max_iter = 200,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

alpha_t = 1
ACS - VRP
fitness = 0.6606055340691207
time = 4.910164833068848
alpha_t = 2
ACS - VRP
fitness = 0.661148855888349
time = 7.235687017440796
alpha_t = 3
ACS - VRP
fitness = 0.6542241217301061
time = 3.6236445903778076
alpha_t = 4
ACS - VRP
fitness = 0.658733138889299
time = 8.140146493911743
alpha_t = 5
ACS - VRP
fitness = 0.6476063947566538
time = 3.8030900955200195


nilai alpha_t terbaik adalah 2, rentang acceptable = [1,2]

### Find beta

In [44]:
#find beta
beta_ls = [1,2,3,4,5]

for beta in beta_ls:
    print(f"beta = {beta}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(init_pheromone=0.1,alpha_t = 1,beta = beta,max_iter = 200,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

beta = 1
ACS - VRP
fitness = 0.6687896412421455
time = 11.814791679382324
beta = 2
ACS - VRP
fitness = 0.6606055340691207
time = 4.725559949874878
beta = 3
ACS - VRP
fitness = 0.6616663755336752
time = 5.7361531257629395
beta = 4
ACS - VRP
fitness = 0.6585768691018593
time = 6.318057060241699
beta = 5
ACS - VRP
fitness = 0.6651968141029742
time = 11.79578709602356


beta 1 atau 5 memiliki hasil lebih baik ketika alpha = 1 tetapi lebih lambat konvergen

acceptable range = alpha [1,2] dan beta = 1 atau 2 atau 5

### Find q0

In [14]:
#find beta
q0_ls = [0.1,0.3,0.5,0.7,0.9]

for q0 in q0_ls:
    print(f"q0 = {q0}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(alpha_t = 1,beta = 1,q0=q0,max_iter = 200,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

q0 = 0.1
ACS - VRP
fitness = 0.6687896412421455
time = 11.08081579208374
q0 = 0.3
ACS - VRP
fitness = 0.6639811903484902
time = 7.48501443862915
q0 = 0.5
ACS - VRP
fitness = 0.6624831060855815
time = 8.814587593078613
q0 = 0.7
ACS - VRP
fitness = 0.6650551225464869
time = 8.973769426345825
q0 = 0.9
ACS - VRP
fitness = 0.6580109882525572
time = 6.427075386047363


q0 = 0.1 memiliki hasil yang baik tetapi lambat konvergen. acceptable value = 0.1 atau 0.7

### Find Rho

In [15]:
#find rho
rho_ls = [0.1,0.3,0.5,0.7,0.9]

for rho in rho_ls:
    print(f"rho = {rho}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(alpha_t = 1,beta = 1,q0=0.1,rho=rho,max_iter = 200,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

rho = 0.1
ACS - VRP
fitness = 0.6687896412421455
time = 11.565764427185059
rho = 0.3
ACS - VRP
fitness = 0.6624348641188019
time = 6.770227670669556
rho = 0.5
ACS - VRP
fitness = 0.6618690204974028
time = 7.656583786010742
rho = 0.7
ACS - VRP
fitness = 0.6640263159621249
time = 4.182241439819336
rho = 0.9
ACS - VRP
fitness = 0.6639748756329066
time = 13.024762630462646


rho 0.1 memberikan hasil baik. acceptable range = [0.1,0.7]

### Find Alpha

In [16]:
#find alpha
alpha_ls = [0.1,0.3,0.5,0.7,0.9]

for alpha in alpha_ls:
    print(f"alpha = {alpha}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(alpha_t = 1,beta = 1,q0=0.1,rho=0.1,alpha=alpha,max_iter = 200,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

alpha = 0.1
ACS - VRP
fitness = 0.6687896412421455
time = 11.074829816818237
alpha = 0.3
ACS - VRP
fitness = 0.6606765730008313
time = 9.202415466308594
alpha = 0.5
ACS - VRP
fitness = 0.6608884558798204
time = 5.642709732055664
alpha = 0.7
ACS - VRP
fitness = 0.6500260314762027
time = 6.14891242980957
alpha = 0.9
ACS - VRP
fitness = 0.6553256856645944
time = 4.98948335647583


semakin besar alpha semakin cepat konvergen tetapi fitnessnya semakin kecil. Hasil terbaik 0.1

### Find Num Ant

In [19]:
#find num_ant
num_ant_ls = [10,30,50]

for num_ant in num_ant_ls:
    print(f"num_ant = {num_ant}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(alpha_t = 1,beta = 1,q0=0.1,rho=0.1,alpha=0.1,num_ant=num_ant,max_iter = 200,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

num_ant = 10
ACS - VRP
fitness = 0.6599400081517238
time = 6.117042541503906
num_ant = 30
ACS - VRP
fitness = 0.6687896412421455
time = 11.313025951385498
num_ant = 50
ACS - VRP
fitness = 0.6598596492954582
time = 7.883904695510864


In [11]:
#find num_ant
num_ant_ls = [10,30,50]

for num_ant in num_ant_ls:
    print(f"num_ant = {num_ant}")
    print("ACS - VRP")
    start = time.time()
    acs_vrp = ACS_VRP(alpha_t = 1,beta = 1,q0=0.1,rho=0.1,alpha=0.1,num_ant=num_ant,max_iter = 200,random_state=100)
    acs_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acs_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

num_ant = 10
ACS - VRP
fitness = 0.6387592698012922
time = 2.9737191200256348
num_ant = 30
ACS - VRP
fitness = 0.6434329363778726
time = 10.601082801818848
num_ant = 50
ACS - VRP
fitness = 0.6441465457591318
time = 20.63413953781128


num_ant terbaik adalah 30

hasil tuning:
- init_pheromone = 0.1 atau 0.3 atau 1
- alpha_t = [1,2]
- beta = 1 atau 5
- q0 = 0.1 atau 0.7
- rho = [0.1,0.7]
- alpha = 0.1
- num_ant = 30

parameter terpilih:
- init_pheromone = 0.1
- alpha_t = 1
- beta = 1
- q0 = 0.1
- rho = 0.1
- alpha = 0.1
- num_ant = 30


### Check TSP

In [22]:
start = time.time()
acs_tsp = ACS_TSP(alpha_t = 1,beta = 1,q0=0.1,rho=0.1,alpha=0.1,num_ant=30,max_iter = 200,random_state=100)
acs_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
fitness = acs_tsp.construct_solution()[-1]
end = time.time()
print(f"fitness = {fitness}")
print(f"time = {end-start}")

fitness = 0.5150052403938413
time = 14.111375570297241


## Tuning BSO (VRP/TSP)
Tuning dilakukan dengan VRP, TSP mengikuti

default parameter/parameter awal:
- p1 = 0.3 untuk VRP | p1 = 0.4 untuk TSP
- p2 = 0.4
- p3 = 0.5
- p4 = 0.5
- two_opt_method = 'first'

### Find p1

In [58]:
#find p1
p1_ls = [0.1,0.3,0.5,0.7,0.9]

for p1 in p1_ls:
    print(f"p1 = {p1}")
    print("BSO - VRP")
    start = time.time()
    bso_vrp = BSO_VRP(p1=p1,max_iter = 200,two_opt_method = "best",random_state=100)
    bso_vrp.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = bso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p1 = 0.1
BSO - VRP
fitness = 0.5691515277511247
time = 2.080500364303589
p1 = 0.3
BSO - VRP
fitness = 0.5612907006092519
time = 0.8220617771148682
p1 = 0.5
BSO - VRP
fitness = 0.5700774536770506
time = 1.122204065322876
p1 = 0.7
BSO - VRP
fitness = 0.5700774536770506
time = 0.5158281326293945
p1 = 0.9
BSO - VRP
fitness = 0.5605190956709802
time = 0.2710599899291992


0.1

### Find p2

In [59]:
#find p2
p2_ls = [0.1,0.3,0.5,0.7,0.9]

for p2 in p2_ls:
    print(f"p2 = {p2}")
    print("BSO - VRP")
    start = time.time()
    bso_vrp = BSO_VRP(p1=0.1,p2=p2,max_iter = 200,two_opt_method = "best",random_state=100)
    bso_vrp.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = bso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p2 = 0.1
BSO - VRP
fitness = 0.5682256018251989
time = 2.056840419769287
p2 = 0.3
BSO - VRP
fitness = 0.5691515277511247
time = 2.061793804168701
p2 = 0.5
BSO - VRP
fitness = 0.5607248569878527
time = 2.048513412475586
p2 = 0.7
BSO - VRP
fitness = 0.5607248569878527
time = 2.0163521766662598
p2 = 0.9
BSO - VRP
fitness = 0.5607248569878527
time = 2.041137456893921


0.3

### Find p3

In [60]:
#find p3
p3_ls = [0.1,0.3,0.5,0.7,0.9]

for p3 in p3_ls:
    print(f"p3 = {p3}")
    print("BSO - VRP")
    start = time.time()
    bso_vrp = BSO_VRP(p1=0.1,p2=0.3,p3=p3,max_iter = 200,two_opt_method = "best",random_state=100)
    bso_vrp.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = bso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p3 = 0.1
BSO - VRP
fitness = 0.5557865853829144
time = 1.31313157081604
p3 = 0.3
BSO - VRP
fitness = 0.5607248569878527
time = 2.1054065227508545
p3 = 0.5
BSO - VRP
fitness = 0.5691515277511247
time = 2.0290749073028564
p3 = 0.7
BSO - VRP
fitness = 0.5612907006092519
time = 2.3685829639434814
p3 = 0.9
BSO - VRP
fitness = 0.567865519520672
time = 1.4327595233917236


0.5

### Find p4

In [61]:
#find p4
p4_ls = [0.1,0.3,0.5,0.7,0.9]

for p4 in p4_ls:
    print(f"p4 = {p4}")
    print("BSO - VRP")
    start = time.time()
    bso_vrp = BSO_VRP(p1=0.1,p2=0.3,p3=0.5,p4=p4,max_iter = 200,two_opt_method = "best",random_state=100)
    bso_vrp.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = bso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p4 = 0.1
BSO - VRP
fitness = 0.5607248569878527
time = 1.231050968170166
p4 = 0.3
BSO - VRP
fitness = 0.5610849392923795
time = 1.4399974346160889
p4 = 0.5
BSO - VRP
fitness = 0.5691515277511247
time = 2.044426679611206
p4 = 0.7
BSO - VRP
fitness = 0.5557865853829144
time = 1.6162891387939453
p4 = 0.9
BSO - VRP
fitness = 0.5525972849713918
time = 0.865875244140625


0.5

### Find p1 TSP

In [6]:
#find p1
p1_ls = [0.1,0.3,0.5,0.7,0.9]

for p1 in p1_ls:
    print(f"p1 = {p1}")
    print("BSO - TSP")
    start = time.time()
    bso_tsp = BSO_TSP(p1=p1,max_iter = 200,two_opt_method = "best",random_state=100)
    bso_tsp.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = bso_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p1 = 0.1
BSO - TSP
fitness = 0.543676775952251
time = 71.09224677085876
p1 = 0.3
BSO - TSP
fitness = 0.543676775952251
time = 60.36173439025879
p1 = 0.5
BSO - TSP
fitness = 0.5415894856705162
time = 54.44775867462158
p1 = 0.7
BSO - TSP
fitness = 0.543676775952251
time = 46.63485932350159
p1 = 0.9
BSO - TSP
fitness = 0.543676775952251
time = 39.93434500694275


0.9

## Tuning Hybrid ACS BSO (VRP/TSP)
ikuti hasil tuning ACS dan BSO

In [32]:
start = time.time()
acsbso_vrp = ACSBSO_VRP(alpha_t = 1,beta = 1,q0=0.1,rho=0.1,alpha=0.1,num_ant=30,max_iter_acs = 200,
                     p1 = 0.4,p2=0.4,p3=0.5,p4=0.5,max_iter_bso=15,random_state=100)
acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
fitness = acsbso_vrp.construct_solution()[-1]
end = time.time()
print(f"fitness = {fitness}")
print(f"time = {end-start}")

fitness = 0.6734238751078129
time = 72.60819482803345


In [12]:
start = time.time()
acsbso_vrp = ACSBSO_VRP(alpha_t = 1,beta = 1,q0=0.1,rho=0.1,alpha=0.1,num_ant=30,max_iter_acs = 200,
                     p1 = 0.4,p2=0.4,p3=0.5,p4=0.5,max_iter_bso=15,random_state=100)
acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
fitness = acsbso_vrp.construct_solution()[-1]
end = time.time()
print(f"fitness = {fitness}")
print(f"time = {end-start}")

fitness = 0.6494000145671729
time = 107.95917701721191


In [13]:
start = time.time()
acsbso_tsp = ACSBSO_TSP(alpha_t = 1,beta = 1,q0=0.1,rho=0.1,alpha=0.1,num_ant=30,max_iter_acs = 200,
                     p1 = 0.9,max_iter_bso=15,random_state=100)
acsbso_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
fitness = acsbso_tsp.construct_solution()[-1]
end = time.time()
print(f"fitness = {fitness}")
print(f"time = {end-start}")

fitness = 0.627314637530019
time = 144.34033846855164


## Tuning ACO (VRP/TSP)
Tuning dilakukan dengan VRP, TSP mengikuti

default parameter/parameter awal:
- alpha_t = 1
- beta = 4
- q0 = 0.6
- init_pheromone = 0.1
- rho = 0.7
- num_ant = 35

### Find Init Pheromone

In [14]:
#find init pheromone
init_pheromone = [0.1,0.3,0.5,1]

for pheromone in init_pheromone:
    print(f"pheromone = {pheromone}")
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = 100,init_pheromone=pheromone,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

pheromone = 0.1
ACO - VRP
fitness = 0.6367464756988616
time = 2.907949686050415
pheromone = 0.3
ACO - VRP
fitness = 0.6301815111688456
time = 4.257891416549683
pheromone = 0.5
ACO - VRP
fitness = 0.6392166770191339
time = 4.698712348937988
pheromone = 1
ACO - VRP
fitness = 0.6419920487889688
time = 4.4397008419036865


pheromone 1 memiliki hasil lebih baik tetapi 0.1 lebih cepat konvergen. Acceptable value = 0.1 atau 1

### Find Alpha_t

In [17]:
#find alpha_t
alpha_t_ls = [1,2,3,4,5]

for alpha_t in alpha_t_ls:
    print(f"alpha_t = {alpha_t}")
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(init_pheromone=1,alpha_t = alpha_t,max_iter = 100,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

alpha_t = 1
ACO - VRP
fitness = 0.6419920487889688
time = 2.909588098526001
alpha_t = 2
ACO - VRP
fitness = 0.6209169418078456
time = 3.710354804992676
alpha_t = 3
ACO - VRP
fitness = 0.6295667490039972
time = 3.5760281085968018
alpha_t = 4
ACO - VRP
fitness = 0.6209169418078456
time = 4.227293252944946
alpha_t = 5
ACO - VRP
fitness = 0.6209169418078456
time = 4.268089771270752


1 memberikan hasil terbaik dengan waktu yang baik juga

### Find beta

In [18]:
#find beta
beta_ls = [1,2,3,4,5]

for beta in beta_ls:
    print(f"beta = {beta}")
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(init_pheromone=1,alpha_t = 1,beta = beta,max_iter = 100,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

beta = 1
ACO - VRP
fitness = 0.6441434088310887
time = 2.943222761154175
beta = 2
ACO - VRP
fitness = 0.6381399595202129
time = 4.1528050899505615
beta = 3
ACO - VRP
fitness = 0.6442297031099563
time = 4.259020090103149
beta = 4
ACO - VRP
fitness = 0.6419920487889688
time = 4.3442771434783936
beta = 5
ACO - VRP
fitness = 0.6299559656495011
time = 4.139296531677246


1 memberikan hasil yang baik dan lebih cepat konvergen

### Find q0

In [19]:
#find beta
q0_ls = [0.1,0.3,0.5,0.7,0.9]

for q0 in q0_ls:
    print(f"q0 = {q0}")
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(init_pheromone = 1,alpha_t = 1,beta = 1,q0=q0,max_iter = 100,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

q0 = 0.1
ACO - VRP
fitness = 0.6577598261592899
time = 4.4432783126831055
q0 = 0.3
ACO - VRP
fitness = 0.6463256658744568
time = 5.40974235534668
q0 = 0.5
ACO - VRP
fitness = 0.6404948051213162
time = 4.162682056427002
q0 = 0.7
ACO - VRP
fitness = 0.638007424060118
time = 3.956200361251831
q0 = 0.9
ACO - VRP
fitness = 0.6349400684526746
time = 3.823956251144409


0.1 memberikan hasil terbaik

### Find Rho

In [20]:
#find rho
rho_ls = [0.1,0.3,0.5,0.7,0.9]

for rho in rho_ls:
    print(f"rho = {rho}")
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(init_pheromone=1,alpha_t = 1,beta = 1,q0=0.1,rho=rho,max_iter = 100,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

rho = 0.1
ACO - VRP
fitness = 0.6491210119851832
time = 3.153700113296509
rho = 0.3
ACO - VRP
fitness = 0.6491210119851832
time = 4.367241382598877
rho = 0.5
ACO - VRP
fitness = 0.6505351823968691
time = 5.156970262527466
rho = 0.7
ACO - VRP
fitness = 0.6577598261592899
time = 6.201971769332886
rho = 0.9
ACO - VRP
fitness = 0.6491210119851832
time = 4.659065246582031


0.7 memberikan hasil terbaik walaupun runningnya cukup lama. Acceptable range = [0.5,0.7]

### Find num_ant

In [21]:
#find num_ant
num_ant_ls = [10,30,35,50]

for num_ant in num_ant_ls:
    print(f"num_ant = {num_ant}")
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(init_pheromone=1,alpha_t = 1,beta = 1,q0=0.1,rho=0.7,num_ant=num_ant,max_iter = 100,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

num_ant = 10
ACO - VRP
fitness = 0.6396014881406362
time = 1.0416336059570312
num_ant = 30
ACO - VRP
fitness = 0.6491210119851832
time = 2.8457512855529785
num_ant = 35
ACO - VRP
fitness = 0.6577598261592899
time = 6.224350452423096
num_ant = 50
ACO - VRP
fitness = 0.6505694930349506
time = 6.884086847305298


35 memberikan hasil terbaik

### Check TSP

In [26]:
start = time.time()
aco_tsp = ACO_TSP(init_pheromone=1,alpha_t = 1,beta = 1,q0=0.1,rho=0.7,num_ant=35,max_iter = 100,random_state=100)
aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
fitness = aco_tsp.construct_solution()[-1]
end = time.time()
print(f"fitness = {fitness}")
print(f"time = {end-start}")

fitness = 0.5052350089665102
time = 8.734225511550903


choosen parameter:
- alpha_t = 1
- beta = 1
- q0 = 0.1
- init_pheromone = 1
- rho = 0.7
- num_ant = 35

## Tuning HACO (VRP/TSP)
Gunakan parameter ACO yang sama dengan ACO_VRP dan tuning parameter terkait mutation-nya

default parameter:
- q1 = 0.6
- q2 = 0.3
- q3 = 0.6

### Find q1

In [27]:
#find q1
q1_ls = [0.1,0.3,0.5,0.6,0.7,0.9]

for q1 in q1_ls:
    print(f"q1 = {q1}")
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(init_pheromone=1,alpha_t = 1,beta = 1,q0=0.1,q1=q1,rho=0.7,num_ant=35,max_iter = 100,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

q1 = 0.1
HACO - VRP
fitness = 0.641174180358118
time = 6.7383198738098145
q1 = 0.3
HACO - VRP
fitness = 0.6544938502957471
time = 13.10752010345459
q1 = 0.5
HACO - VRP
fitness = 0.6613771390073923
time = 15.296005725860596
q1 = 0.6
HACO - VRP
fitness = 0.6614851144931745
time = 12.40409517288208
q1 = 0.7
HACO - VRP
fitness = 0.662174464110273
time = 16.132529497146606
q1 = 0.9
HACO - VRP
fitness = 0.6626160431500097
time = 18.687308311462402


0.6 memberikan hasil cukup baik dengan waktu running sedikit

### Find q2

In [28]:
#find q2
q2_ls = [0.1,0.3,0.5,0.6,0.7,0.9]

for q2 in q2_ls:
    print(f"q2 = {q2}")
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(init_pheromone=1,alpha_t = 1,beta = 1,q0=0.1,q1=0.6,q2=q2,rho=0.7,num_ant=35,max_iter = 100,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

q2 = 0.1
HACO - VRP
fitness = 0.6618083346994356
time = 13.632583141326904
q2 = 0.3
HACO - VRP
fitness = 0.6614851144931745
time = 11.849156379699707
q2 = 0.5
HACO - VRP
fitness = 0.6639799704602525
time = 15.419055938720703
q2 = 0.6
HACO - VRP
fitness = 0.6565192262587196
time = 14.914613246917725
q2 = 0.7
HACO - VRP
fitness = 0.6608308939884857
time = 14.804694175720215
q2 = 0.9
HACO - VRP
fitness = 0.661148855888349
time = 15.03340196609497


0.3 memberikan hasil cukup baik dengan running time kecil namun 0.5 memberikan hasil terbaik. acceptable range = [0.1,0.5]

### Find q3

In [29]:
#find q3
q3_ls = [0.1,0.3,0.5,0.6,0.7,0.9]

for q3 in q3_ls:
    print(f"q3 = {q3}")
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(init_pheromone=1,alpha_t = 1,beta = 1,q0=0.1,q1=0.6,q2=0.5,q3=q3,rho=0.7,num_ant=35,max_iter = 100,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

q3 = 0.1
HACO - VRP
fitness = 0.6603258106208595
time = 13.30063009262085
q3 = 0.3
HACO - VRP
fitness = 0.6640136865309576
time = 12.632752418518066
q3 = 0.5
HACO - VRP
fitness = 0.660162244164456
time = 13.131663799285889
q3 = 0.6
HACO - VRP
fitness = 0.6639799704602525
time = 15.225099325180054
q3 = 0.7
HACO - VRP
fitness = 0.6575994731722998
time = 11.470699548721313
q3 = 0.9
HACO - VRP
fitness = 0.6598536021891475
time = 14.568622350692749


0.3 memberikan hasil terbaik dan running time yang acceptable

### Check TSP

In [30]:
start = time.time()
haco_tsp = HACO_TSP(init_pheromone=1,alpha_t = 1,beta = 1,q0=0.1,q1=0.6,q2=0.5,q3=0.3,rho=0.7,num_ant=35,max_iter = 100,random_state=100)
haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
fitness = haco_tsp.construct_solution()[-1]
end = time.time()
print(f"fitness = {fitness}")
print(f"time = {end-start}")

fitness = 0.5614452579766339
time = 16.4608952999115


choosen parameter:
- q1 = 0.6
- q2 = 0.3
- q3 = 0.3

In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import threading
import random
import datetime
import copy

In [23]:
def get_var(strs):
    t = str(strs)
    print(t)
    tourid = []
    t_ = json.loads(t)
    for t in t_['idWisata']:
        tourid.append(t['id'])
    idhotel = int(t_['idhotel'][0])
    dwaktu = float(t_['degree'][0])
    dtarif = float(t_['degree'][2])
    drating = float(t_['degree'][1])
    travel_days = int(t_['travel_days'])
    return tourid,idhotel,dwaktu,dtarif,drating,travel_days

choice = '{"idWisata":[{"id":"1"},{"id":"2"},{"id":"3"},{"id":"4"},{"id":"5"},{"id":"6"},{"id":"7"},{"id":"8"},{"id":"9"},{"id":"10"},{"id":"11"},{"id":"12"},{"id":"13"},{"id":"14"},{"id":"15"},{"id":"16"},{"id":"17"},{"id":"18"},{"id":"19"},{"id":"20"},{"id":"21"},{"id":"22"},{"id":"23"},{"id":"24"},{"id":"25"},{"id":"26"},{"id":"27"},{"id":"28"},{"id":"29"},{"id":"69"}],"idhotel":["169"],"algo_url":"ant-colony-optimization-aco-tsp","degree":[1,1,1],"travel_days":6}'
tourid,idhotel,dwaktu,dtarif,drating,travel_days = get_var(choice)

{"idWisata":[{"id":"1"},{"id":"2"},{"id":"3"},{"id":"4"},{"id":"5"},{"id":"6"},{"id":"7"},{"id":"8"},{"id":"9"},{"id":"10"},{"id":"11"},{"id":"12"},{"id":"13"},{"id":"14"},{"id":"15"},{"id":"16"},{"id":"17"},{"id":"18"},{"id":"19"},{"id":"20"},{"id":"21"},{"id":"22"},{"id":"23"},{"id":"24"},{"id":"25"},{"id":"26"},{"id":"27"},{"id":"28"},{"id":"29"},{"id":"69"}],"idhotel":["169"],"algo_url":"ant-colony-optimization-aco-tsp","degree":[1,1,1],"travel_days":6}


In [24]:
db = ConDB()
rute_perhari = []
waktuDatang = []

start = time.time()

hotel = db.HotelbyID(idhotel)
tur = db.WisatabyID(tourid)
timematrix = db.TimeMatrixbyID(hotel._id,tourid)

In [25]:
start = time.time()
acs_model = ACS_VRP(max_iter=200,max_idem=30,num_ant=10,random_state = 100)
acs_model.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)

sol,fitness = acs_model.construct_solution()
end = time.time()
run = end-start
print(fitness)
print(run)

0.6611956364097826
2.0480401515960693


In [26]:
sol

[{'index': [13, 18, 2, 69, 12, 4, 5, 21, 27],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 11),
   datetime.time(9, 22),
   datetime.time(10, 32),
   datetime.time(11, 35),
   datetime.time(12, 45),
   datetime.time(14, 30),
   datetime.time(16, 12),
   datetime.time(17, 15),
   datetime.time(18, 14),
   datetime.time(20, 34)],
  'rating': [4.8, 4.5, 4.7, 4.6, 4.7, 4.6, 4.5, 4.6, 4.5],
  'tarif': [0, 0, 0, 8000, 3000, 0, 35000, 0, 3000]},
 {'index': [17, 6, 11, 3, 14, 20, 7, 10, 9, 1],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 18),
   datetime.time(9, 13),
   datetime.time(11, 17),
   datetime.time(12, 5),
   datetime.time(12, 48),
   datetime.time(13, 37),
   datetime.time(15, 24),
   datetime.time(16, 25),
   datetime.time(17, 27),
   datetime.time(18, 8),
   datetime.time(19, 49)],
  'rating': [4.5, 4.6, 4.5, 4.6, 4.5, 4.7, 4.8, 4.6, 4.7, 4.8],
  'tarif': [0, 3000, 0, 0, 0, 10000, 0, 0, 0, 0]},
 {'index': [8, 24, 22, 28, 15, 23, 26],
  'waktu': [datetime.time(8, 0

In [29]:
start = time.time()
acs_model = ACS_TSP(max_iter=200,random_state = 100)
acs_model.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)

sol,fitness = acs_model.construct_solution()
end = time.time()
run = end-start
print(fitness)
print(run)

0.6427285582204815
12.973615884780884


In [30]:
sol

[{'index': [1, 5, 10, 7, 9],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 13),
   datetime.time(16, 0),
   datetime.time(16, 49),
   datetime.time(17, 50),
   datetime.time(18, 51),
   datetime.time(19, 35)],
  'rating': [4.8, 4.5, 4.6, 4.8, 4.7],
  'tarif': [0, 35000, 0, 0, 0]},
 {'index': [8, 4, 23, 14, 27, 28],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 12),
   datetime.time(10, 17),
   datetime.time(11, 52),
   datetime.time(12, 32),
   datetime.time(13, 46),
   datetime.time(16, 12),
   datetime.time(18, 56)],
  'rating': [4.7, 4.6, 4.6, 4.5, 4.5, 4.4],
  'tarif': [3000, 0, 0, 0, 3000, 115000]},
 {'index': [20, 22, 12, 13, 2, 3, 17, 24],
  'waktu': [datetime.time(8, 0),
   datetime.time(9, 0),
   datetime.time(10, 50),
   datetime.time(12, 21),
   datetime.time(13, 52),
   datetime.time(14, 59),
   datetime.time(16, 6),
   datetime.time(16, 38),
   datetime.time(17, 47),
   datetime.time(18, 41)],
  'rating': [4.7, 4.5, 4.7, 4.8, 4.7, 4.6, 4.5, 4.4],
  'tarif': [

In [12]:
len(sum([i['index'] for i in sol],[]))

28

In [7]:
start = time.time()
acsbso_model = ACSBSO_VRP(max_iter_acs=200,random_state = 100)
acsbso_model.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)

sol,fitness = acsbso_model.construct_solution()
end = time.time()
run = end-start
print(fitness)
print(run)

0.6636147933283799
40.4247465133667


In [ ]:
acsbso_model

In [5]:
start = time.time()
acsbso_model = ACSBSO_TSP(max_iter_acs=300,max_idem_acs=30,max_iter_bso=15,p1=0.7, random_state = 100)
acsbso_model.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)

sol,fitness = acsbso_model.construct_solution()
end = time.time()
run = end-start
print(fitness)
print(run)

0.5199885335653813
396.80543303489685


In [7]:
sol

[{'index': [6, 14, 26, 24, 22, 5, 10, 13, 9],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 16),
   datetime.time(10, 35),
   datetime.time(11, 32),
   datetime.time(13, 49),
   datetime.time(14, 33),
   datetime.time(16, 9),
   datetime.time(16, 58),
   datetime.time(18, 9),
   datetime.time(19, 19),
   datetime.time(20, 3)],
  'rating': [4.6, 4.5, 4.5, 4.4, 4.5, 4.5, 4.6, 4.8, 4.7],
  'tarif': [3000, 0, 0, 0, 0, 35000, 0, 0, 0]},
 {'index': [29, 20, 27, 21, 15, 17, 3, 4],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 53),
   datetime.time(11, 26),
   datetime.time(13, 38),
   datetime.time(15, 52),
   datetime.time(16, 18),
   datetime.time(16, 52),
   datetime.time(17, 39),
   datetime.time(18, 9),
   datetime.time(19, 54)],
  'rating': [4.3, 4.7, 4.5, 4.6, 4.5, 4.5, 4.6, 4.6],
  'tarif': [30000, 10000, 3000, 0, 0, 0, 0, 0]},
 {'index': [16, 12, 69, 11, 1, 2, 7, 23],
  'waktu': [datetime.time(8, 0),
   datetime.time(8, 9),
   datetime.time(9, 26),
   datetime.time(11, 

In [4]:
bso_model = BSO_VRP(random_state = 100)
bso_model.set_model(tur,hotel,timematrix,init_solution=[],travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)

sol_node,sol,fitness = bso_model.construct_solution()
fitness

0.6603854512245971

In [22]:
command = "BSO_VRP" + "(max_iter={iteration},random_state=10)".format(iteration="10")
exec("model = {command}".format(command=command))

In [9]:
random.seed(None)

In [6]:
a = [0.3791177272796631,
0.43644261360168457,
0.38830065727233887,
0.6996159553527832,
0.46743106842041016,
0.3434772491455078,
0.5800998210906982,
0.4150519371032715,
0.44530248641967773,
0.6571595668792725,
0.40494465827941895,
0.692896842956543,
0.8342714309692383,
0.6910789012908936,
0.5129132270812988,
0.4804236888885498,
0.3209686279296875,
0.42601847648620605,
0.39066433906555176,
0.5672774314880371,
0.9084668159484863,
0.27115368843078613,
0.40529322624206543,
0.7366936206817627,
0.594475269317627,
0.4014918804168701,
0.41829848289489746,
0.8024146556854248,
0.6096019744873047,
0.4448666572570801,
0.8894081115722656,
0.16657662391662598,
0.5153968334197998,
0.8408451080322266,
0.29433155059814453,
0.2928009033203125,
0.4803731441497803,
0.4021484851837158,
0.3879678249359131,
0.6936070919036865,
0.4454841613769531,
0.7051844596862793,
0.43483638763427734,
0.4561753273010254,
0.649989128112793,
0.44147682189941406,
0.548473596572876,
0.33518552780151367,
0.5097179412841797,
0.6174004077911377,
0.6508307456970215,
0.5080409049987793,
1.111036777496338,
0.8744893074035645,
0.7643203735351562,
0.45719456672668457,
0.45942211151123047,
0.5107338428497314,
0.39502978324890137,
0.6055765151977539,
0.36304354667663574,
0.49088478088378906,
0.6541593074798584,
0.703129768371582,
0.30786871910095215,
0.6256604194641113,
0.4805271625518799,
0.5253174304962158,
0.1745436191558838,
0.4888637065887451,
0.3309011459350586,
0.4515209197998047,
0.43791770935058594,
0.20480680465698242,
0.5752556324005127,
0.5809659957885742,
0.6340129375457764,
0.5782098770141602,
0.4679274559020996,
0.4718747138977051,
0.49535179138183594,
0.6401042938232422,
0.5307507514953613,
0.48633623123168945,
0.33688807487487793,
0.6628160067801528]

In [7]:
sum(a)

44.86990614616919